In [5]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
!kaggle datasets download -d borismarjanovic/price-volume-data-for-all-us-stocks-etfs

Dataset URL: https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
License(s): CC0-1.0
 96% 473M/492M [00:02<00:00, 247MB/s]
100% 492M/492M [00:02<00:00, 204MB/s]


In [12]:
import zipfile
zip_ref = zipfile.ZipFile('price-volume-data-for-all-us-stocks-etfs.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
import os
os.getcwd()

'/content'

In [39]:
import pandas as pd
from pandas.errors import EmptyDataError

dataframes = []
directory = os.getcwd() + '/Stocks'

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        ticker = filename.replace('.us.txt', '')
        file_path = os.path.join(directory, filename)

        try:
            df = pd.read_csv(file_path)
            df['Ticker'] = ticker
            df.sort_values('Date', inplace=True)
            dataframes.append(df)
        except EmptyDataError: # some of the files have no contents so read_csv throws error
          pass

stock_df = pd.concat(dataframes, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'], format='%Y-%m-%d')

In [40]:
stock_df.tail()

,Date,Open,High,Low,Close,Volume,OpenInt,Ticker
14887660,2017-10-17,25.1900,25.2100,25.1800,25.1800,7505,0,star_e
14887661,2017-10-18,25.1800,25.2700,25.1800,25.1920,7572,0,star_e
14887662,2017-10-19,25.2316,25.2413,25.2316,25.2413,320,0,star_e
14887663,2017-10-20,25.2400,25.2400,24.5000,25.2400,1120,0,star_e
14887664,2017-10-23,25.3500,25.3500,25.3500,25.3500,1542,0,star_e


In [30]:
from os import listdir
from os.path import isfile, join

tickers = set([file.split(".")[0] for file in listdir("Stocks")])

In [35]:
TRAINING_START_DATE = "2005-01-01"
TRAINING_END_DATE = "2012-12-31"

In [34]:
!pip install ta

In [41]:
import ta

i = 0
dataframes = []
for ticker in tickers:
    i += 1
    if i % 1000 == 0:
        print(i)
    ticker_df = stock_df[stock_df['Ticker'] == ticker & stock_df['Date'] >= TRAINING_START_DATE & stock_df['Date'] <= TRAINING_END_DATE]

    dataframes.append(ta.add_all_ta_features(ticker_df, open="Open", high="High", low="Low", close="Close", volume="Volume"))

train_df = pd.concat(dataframes, ignore_index=True)

TypeError: unsupported operand type(s) for &: 'str' and 'DatetimeArray'